In [46]:
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score 
import pickle
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer  
import spacy
from sklearn import *
nlp=spacy.load('es_core_news_lg')
spanish_stemmer = SnowballStemmer('spanish')
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn import linear_model
from sklearn import *
from sklearn.metrics import *
from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers
from sklearn.model_selection import train_test_split
from keras.initializers import glorot_uniform  # Or your initializer of choice
import keras.backend as K
from keras.models import load_model
import matplotlib.pyplot as plt

In [47]:
#importamos datos
df= pd.read_csv('data/intent.csv',sep=';',encoding='UTF-8') 
df=df.drop(columns="Unnamed: 5")
oraciones=df.drop(axis=1,columns=["id","intencion","sub_intencion"])
intencion=df.drop(axis=1,columns=['id','sub_intencion','oracion'])
sub_intencion=df.drop(axis=1,columns=["id","intencion","oracion"])

sub_intencion.head(2)

,sub_intencion,accion
0,google,buscar_google
1,google,buscar_google


In [48]:
def stopWords(corpus):
    global oraciones_filtradas
    f = open('stopwords.txt', 'r')
    stopwords = f.read().split('\n')
    f.close()
    " ".join(stopwords)
    oraciones=list(corpus)

    oraciones_filtradas = [" ".join([
                              palabra for palabra in oracion.split()
                              if palabra not in stopwords])
                              for oracion in oraciones]

In [49]:
stopWords(df['oracion'])

In [50]:
oraciones_filtradas_lematizadas=[]
for oracion in oraciones_filtradas:
    oracion_lematizada = []
    for palabra in oracion.split():
        doc=nlp(palabra)
        oracion_lematizada.append(doc[0].lemma_)
    oraciones_filtradas_lematizadas.append(" ".join(oracion_lematizada))
    

oraciones_filtradas_lematizadas_stemm=[]

for oracion in oraciones_filtradas_lematizadas:
    oracion_stemm = []
    for palabra in oracion.split():
        oracion_stemm.append(spanish_stemmer.stem(palabra))
    oraciones_filtradas_lematizadas_stemm.append(" ".join(oracion_stemm))
    
df['oracion'] = oraciones_filtradas_lematizadas_stemm
df = df.drop_duplicates().reset_index(drop=True)
df=df.drop(columns="id")

In [51]:
df1=df.drop(columns="intencion")
df1=df1.drop(columns="sub_intencion")

In [52]:
df1

,oracion,accion
0,busc googl,buscar_google
1,busc,buscar_google
2,busc internet,buscar_google
3,quer busc,buscar_google
4,googl,buscar_google
...,...,...
219,yo ir qued sord,volumenMenos
220,el volum alto,volumenMenos
221,bajal volum,volumenMenos
222,baj poc volum,volumenMenos


In [53]:

    from sklearn.feature_extraction.text import CountVectorizer  
    global carreras_listas
    global carreras_listo
    corpus=df1['oracion']

 
    vectorizador = CountVectorizer(binary=False, ngram_range=(1,2))  
    X = vectorizador.fit_transform(corpus)

    pickle.dump(vectorizador.vocabulary_,open("carreras.pkl","wb"))
    vocabulario = vectorizador.get_feature_names()

    carreras_listas=pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())


    #TF IDF TRANSFORM
    transformer = TfidfTransformer()  
    carreras_listas =transformer.fit_transform(carreras_listas) 

    #lo paso a dataframe
    carreras_listas=pd.DataFrame(carreras_listas.toarray(), columns=vectorizador.get_feature_names())
    #le agrego la columna de oracion y carrera
    oracion_listo=df1.join(carreras_listas)



C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [54]:
carreras_listo

,oracion,accion,abrir,abrir alarm,abrir calculador,abrir carpet,abrir chrom,abrir configur,abrir googl,abrir naveg,...,what,what tim,wiki,wikipd,wikipedi,windows,www,yo,yo ir,youtub
0,busc googl,buscar_google,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,busc,buscar_google,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,busc internet,buscar_google,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,quer busc,buscar_google,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,googl,buscar_google,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,yo ir qued sord,volumenMenos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347884,0.395825,0.0
220,el volum alto,volumenMenos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
221,bajal volum,volumenMenos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
222,baj poc volum,volumenMenos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


In [67]:
 y

0      buscar_google
1      buscar_google
2      buscar_google
3      buscar_google
4      buscar_google
           ...      
219     volumenMenos
220     volumenMenos
221     volumenMenos
222     volumenMenos
223          new_tab
Name: accion, Length: 224, dtype: object

In [55]:
X=  oracion_listo.drop(axis=1,columns=['oracion','accion'])
y=df1["accion"]
"""from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
y=lb.fit_transform(y)
"""

'from sklearn.preprocessing import LabelEncoder\nlb=LabelEncoder()\ny=lb.fit_transform(y)\n'

In [56]:
df1["accion"].unique()


array(['buscar_google', 'buscar_wikipedia', 'buscar_youtube', 'pausa',
       'play', 'anterior', 'siguiente', 'new_tab', 'minimizar',
       'maximizar', 'next_tab', 'previus_tab', 'close_tab', 'previus_pag',
       'new_windows', 'abir_web', 'abrir_carpeta', 'configuraciones',
       'alarma', 'calculadora', 'reloj', 'clima', 'noticias',
       'volumenMas', 'volumenMenos'], dtype=object)

In [57]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)

In [58]:
Knn= neighbors.KNeighborsClassifier(n_neighbors=4,weights='distance')
Knn.fit(X_train, y_train.ravel())
y_pred3= Knn.predict(X_test)
ACkNN=accuracy_score(y_test, y_pred3)

print("Accuracy: ", round(ACkNN,4))

Accuracy:  0.7778


In [59]:
modeloRLog=linear_model.LogisticRegression(max_iter= 5000,penalty='none',fit_intercept=False, random_state=123)
modeloRLog.fit(X_train,y_train.ravel())
y_pred=modeloRLog.predict(X_test)

AC_RLog=accuracy_score(y_test, y_pred)
print("Accuracy: ", round(AC_RLog,3))

Accuracy:  0.889


In [60]:
#RED NEURONAL
Resumen=[]     
funAct=["sigmoid","relu","selu","softmax","softplus","elu","tanh"]
#funAct=["relu"]
for i in funAct: 
    modelo = Sequential()
    modelo.add(Input(shape=(320,), name= "Entrada"))
    print("FUNCION DE ACTIVACION: ====> ",[i])
    
    modelo.add(Dense(units=100, activation=i,kernel_initializer=initializers.glorot_uniform(seed=123)))
    modelo.add(Dense(units=80, activation=i,kernel_initializer=initializers.glorot_uniform(seed=123)))
    modelo.add(Dense(units=50, activation=i,kernel_initializer=initializers.glorot_uniform(seed=123)))

    
    #capa de salida
    modelo.add(Dense(units=224, activation='sigmoid',kernel_initializer=initializers.glorot_uniform(seed=123)))
    #compilamos
    modelo.compile(loss= 'sparse_categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
 
    #modelo.save_weights('model.h5')
    #initial_weights = modelo.get_weights()

    #Entrenamos y hacemos prediccion
    modelo.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=50,batch_size=16)
    #modelo.fit(X,y,validation_split=0.25, epochs=32,batch_size=32)
    #modelo.fit(X,y,epochs=32,batch_size=32)
    
    df_history= pd.DataFrame(modelo.history.history)
    array_history=np.array(df_history)
    
    print("BEST AC "+  str("{:.4f}".format(df_history.val_accuracy.max())) + " --- VAL LOSS: "+str("{:.4f}".format(df_history.val_loss.min())))
    print("EPOCH: "+ str(np.argmax(array_history[:,3])))
    Resumen.append(i + " - BEST AC "+  str("{:.4f}".format(df_history.val_accuracy.max())) + " --- VAL LOSS: "+str("{:.4f}".format(df_history.val_loss.min()))+ " - EPOCH: "+ str(np.argmax(array_history[:,3])))
    print("___________________________________________________________________________________")
    #modelo.load_weights('model.h5')
    #modelo.set_weights(initial_weights)
   

FUNCION DE ACTIVACION: ====>  ['sigmoid']
Epoch 1/50


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "C:\Users\lobo\anaconda3\envs\IA\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\lobo\anaconda3\envs\IA\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\lobo\anaconda3\envs\IA\lib\asyncio\base_events.py", line 595, in run_fo

In [61]:
Resumen

[]

In [62]:
"""import pandas as pd

df_history=pd.DataFrame(modelo.history.history)
"""

'import pandas as pd\n\ndf_history=pd.DataFrame(modelo.history.history)\n'

In [63]:
"""import matplotlib.pyplot as plt
df_history.plot(figsize=(12, 8),grid=True, title="Learning Curves",ylim=(0,1.5));"""

'import matplotlib.pyplot as plt\ndf_history.plot(figsize=(12, 8),grid=True, title="Learning Curves",ylim=(0,1.5));'

In [64]:
"""produccion1=Knn.fit(X,y)
nombre_archivo='modelo_accion2.sav'
pickle.dump(produccion1, open(nombre_archivo, 'wb'))"""

"produccion1=Knn.fit(X,y)\nnombre_archivo='modelo_accion2.sav'\npickle.dump(produccion1, open(nombre_archivo, 'wb'))"

In [65]:
df1["accion"].unique()[4]

'play'

In [66]:
produccion2=modeloRLog.fit(X,y)
nombre_archivo='modelo_accion2.sav'
pickle.dump(produccion2, open(nombre_archivo, 'wb'))

In [ ]:
"""nuevo="quer escuch stast youtub"
produccion1.predict(nuevo)"""

-----------------------------------------------------------------------

In [ ]:
"""Knn= neighbors.KNeighborsClassifier(n_neighbors=5,weights='distance')
Knn.fit(X_train, y_train.ravel())
y_pred3= Knn.predict(X_test)
ACkNN=accuracy_score(y_test, y_pred3,normalize=True)
F1kNN=f1_score(y_test,y_pred3, average='macro')

        
print('Valor de k: ', k_max)
print("Accuracy: ", round(ACkNN_max,3))"""

In [ ]:
"""modeloRLog=linear_model.LogisticRegression(max_iter= 5000,penalty='none',fit_intercept=False, random_state=123)
modeloRLog.fit(X_train,y_train.ravel())
y_pred=modeloRLog.predict(X_test)

AC_RLog=accuracy_score(y_test, y_pred)
print("Accuracy: ", round(AC_RLog,3))"""

-----------------------------------------------

In [ ]:
"""produccion1=Knn.fit(X,y)
nombre_archivo='modelo_accion.sav'
pickle.dump(produccion1, open(nombre_archivo, 'wb'))"""